In [1]:
import ipopt
import numpy as np
import scipy.sparse as sps


class hs071(object):
    def __init__(self):
        pass

    def objective(self, x):
        #
        # The callback for calculating the objective
        #
        return x[0] * x[3] * np.sum(x[0:3]) + x[2]

    def gradient(self, x):
        #
        # The callback for calculating the gradient
        #
        return np.array([
                    x[0] * x[3] + x[3] * np.sum(x[0:3]),
                    x[0] * x[3],
                    x[0] * x[3] + 1.0,
                    x[0] * np.sum(x[0:3])
                    ])

    def constraints(self, x):
        #
        # The callback for calculating the constraints
        #
        return np.array((np.prod(x), np.dot(x, x)))

    def jacobian(self, x):
        #
        # The callback for calculating the Jacobian
        #
        return np.concatenate((np.prod(x) / x, 2*x))

    def hessianstructure(self):
        #
        # The structure of the Hessian
        # Note:
        # The default hessian structure is of a lower triangular matrix. Therefore
        # this function is redundant. I include it as an example for structure
        # callback.
        #
        global hs

        hs = sps.coo_matrix(np.tril(np.ones((4, 4))))
        return (hs.col, hs.row)

    def hessian(self, x, lagrange, obj_factor):
        #
        # The callback for calculating the Hessian
        #
        H = obj_factor*np.array((
                (2*x[3], 0, 0, 0),
                (x[3],   0, 0, 0),
                (x[3],   0, 0, 0),
                (2*x[0]+x[1]+x[2], x[0], x[0], 0)))

        H += lagrange[0]*np.array((
                (0, 0, 0, 0),
                (x[2]*x[3], 0, 0, 0),
                (x[1]*x[3], x[0]*x[3], 0, 0),
                (x[1]*x[2], x[0]*x[2], x[0]*x[1], 0)))

        H += lagrange[1]*2*np.eye(4)

        #
        # Note:
        #
        #
        return H[hs.row, hs.col]

    def intermediate(
            self,
            alg_mod,
            iter_count,
            obj_value,
            inf_pr,
            inf_du,
            mu,
            d_norm,
            regularization_size,
            alpha_du,
            alpha_pr,
            ls_trials
            ):

        #
        # Example for the use of the intermediate callback.
        #
        print('Objective value at iteration #%d is - %g',iter_count, obj_value)


x0 = [1.0, 5.0, 5.0, 1.0]

lb = [1.0, 1.0, 1.0, 1.0]
ub = [5.0, 5.0, 5.0, 5.0]

cl = [25.0, 40.0]
cu = [2.0e19, 40.0]

nlp = ipopt.problem(
            n=len(x0),
            m=len(cl),
            problem_obj=hs071(),
            lb=lb,
            ub=ub,
            cl=cl,
            cu=cu
            )
x, info = nlp.solve(x0)

def __init__(self):
        #     self.N = N
        #     self.nStates = nStates
        #     self. weightDist = weightDist
        #     self. weightVel = weightVel
        #     self. weightTheta = weightTheta
        #     self. weightTime = weightTime
        #     self.xOuter = xOuter
        #     self.yOuter = yOuter
        #     self.xInner = xInner
        #     self.yInner = yInner
        #     self.A = object.A
        #     self.B = object.B
        #     self.f = 0
        #     self.k = 1
        #     self.x = x0 #####FIX
        #
        # def objective(self):
        #     for i in range(self.N):
        #         self.f = self.f+ self.weightDist*(self.x[self.k]-self.x[self.k+self.nStates])**2\
        #                  +self.weightDist*(self.x[self.k+1]-self.x[self.k+1+self.nStates])**2\
        #                         +self.weightVel*(self.x[self.k+2]-self.x[self.k+2+self.nStates])**2\
        #                                 +self.weightTheta*(self.x[self.k+3]-self.x[self.k+3+self.nStates])**2\
        #                                         +self.weightTime*self.x[self.k+10]
        #         self.k=self.k+nStates
        #
        # def gradient(self):
        #     self.g=np.zeros((self.N+1)*self.nStates,1)
        #     for i in range(self.N):
        #         if i==1:
        #             self.g[self.m,1] = 2*self.weightDist*(self.x[self.k]-self.x[self.k+self.nStates])
        #             self.g[self.m+1, 1] = 2 * self.weightDist * (self.x[self.k+1] - self.x[self.k+1 + self.nStates])
        #             self.g[self.m + 2, 1] = 2 * self.weightDist * (\
        #                         self.x[self.k + 2] - self.x[self.k + 2 + self.nStates])
        #             self.g[self.m+3,1] =2*self.weightTheta*(self.x[self.k+3]-self.x[self.k+3+nStates])
        #             self.g[self.m+10,1]=self.weightTime
        #         elif i==self.N+1:
        #             self.g[self.m,1] = -2*weightDist*(self.x[self.k-self.nStates]-self.x[self.k])
        #             self.g[self.m+1, 1] = -2 * weightDist * (self.x[self.k+1- self.nStates] - self.x[self.k+1])
        #             self.g[self.m + 2, 1] = -2 * self.weightDist * ( \
        #                        self.x[self.k + 2- self.nStates] - self.x[self.k + 2 ])
        #             self.g[self.m + 3, 1] = 2 * self.weightTheta * (self.x[self.k + 3- nStates] - self.x[self.k + 3])
        #             self.g[self.m + 10, 1] = self.weightTime
        #         else:
        #             self.g[self.m,1] = -2*self.weightDist*(self.x[self.k-self.nStates]-self.x[self.k])+\
        #                                2*self.weightDist*(self.x[self.k]-self.x[self.k+self.nStates])
        #             self.g[self.m+1,1] = -2*self.weightDist*(self.x[self.k+1 - self.nStates]-self.x[self.k+1])+\
        #                                  2*self.weightDist*(self.x[self.k+1]-self.x[self.k+1+self.nStates])
        #             self.g[self.m+2,1] = -2*self.weightDist*(self.x[self.k+2 - self.nStates]-self.x[self.k+2])+\
        #                                  2*self.weightDist*(self.x[self.k+2]-self.x[self.k+2+self.nStates])
        #             self.g[self.m+3,1] = -2*self.weightDist*(self.x[self.k+3 - self.nStates]-self.x[self.k+3])+ \
        #                                  2 * self.weightDist * (self.x[self.k + 3] - self.x[self.k + 3 + self.nStates])
        #         self.m = self.m+self.nStates
        #         self.k = self.k+self.nStates
        #
        # def constraints(self):
        #     self.nConstraintsSystem = 4
        #     self.nConstraintsTriplet = 1
        #     self.nConstraintsObject = 2
        #     self.nConstraints = self.nConstraintsTriplet+self.nConstraintsSystem+self.nConstraintsObject
        #
        #     self.c = np.zeros((self.nConstraints,1))
        #
        #     ###line 186

SyntaxError: unexpected EOF while parsing (Temp/ipykernel_3116/4234987565.py, line 179)